In [1]:
import numpy as np
import signal
import time
import os
from tqdm import tqdm

Define book object:


In [2]:
class Book:
    def __init__(self, type, points):
        self.book_id = type         # There are B different books with IDs from 0 to B–1
        self.score = points         # the score that is awarded when the book is scanned.

    def __str__(self):
        return "\nBook ID: %d \n " \
               "Book score: %d \n"  % \
               (self.book_id, self.score)

    def __eq__(self, other):
        if not other:
            return False
        return self.book_id == other.book_id

    def __hash__(self):
        return hash((self.book_id,self.score))

Define Library object:

In [3]:
class Library:
    def __init__(self, type, books, time, number):
        self.library_id = type                  # There are L different libraries with IDs from 0 to L–1.
        self.set_of_books = books               # the set of books in the library,
        self.sign_up_time = time                # the time in days that it takes to sign the library up for scanning,
        self.books_per_day = number             # the number of books that can be scanned each day from the library once the library is signed up
        self.num_of_books = len(self.set_of_books)

    def __str__(self):
        return "Library ID: %d\n " \
               "List of books: %s\n " \
               "Time to sign up: %d\n " \
               "Books per day: %d" % \
               (self.library_id,
                ''.join(str(i) for i in self.set_of_books),
                self.sign_up_time,
                self.books_per_day)

Read file function:

In [4]:
def read_from_file(file_name):

    file = open(file_name, 'r').readlines()

    num_of_books, num_of_libraries, max_time = map(int, file[0].split())
    scores_of_books = list(map(int, file[1].split()))
    libraries = []

    for line in range(1,len(file)//2):
        num_of_books_in_lib, sign_up_time, books_per_day = map(int, file[line*2].split())
        books_in_library = set(map(int, file[line*2+1].split()))

        books_in_lib = [Book(idx,scores_of_books[idx]) for idx in books_in_library]
        libraries.append(Library(line, books_in_lib, sign_up_time, books_per_day))

    return libraries,num_of_books, num_of_libraries, max_time


Get best books from library:

In [5]:
def best_books(lib, day,SCANED_BOOKS):
    time = MAX_TIME - lib.sign_up_time - day
    sorted_books = sorted([book for book in lib.set_of_books if book not in SCANED_BOOKS], key=lambda x:x.score, reverse=True)

    return list(sorted_books)[:time*lib.books_per_day]

Count score from best books:


In [6]:
def count_score(lib, day,SCANED_BOOKS,USED_LIBS):
    if lib in USED_LIBS:
        return float('-inf')

    new_books = best_books(lib,day,SCANED_BOOKS)

    score = sum(book.score for book in new_books)
    return score/lib.sign_up_time

Main solving code:

In [7]:
def solve(SCANED_BOOKS,USED_LIBS):
    day = 0
    for library in tqdm(range(NUM_OF_LIBS)):
        scores = [count_score(lib,day,SCANED_BOOKS,USED_LIBS) for lib in LIBRARIES]
        best_lib = np.argmax(scores)

        if LIBRARIES[best_lib] in USED_LIBS:
            break

        new_best_books = best_books(LIBRARIES[best_lib],day,SCANED_BOOKS)

        day += LIBRARIES[best_lib].sign_up_time
        if day >= MAX_TIME:
            break

        SCANED_BOOKS = SCANED_BOOKS.union(set(new_best_books))
        USED_LIBS.add(LIBRARIES[best_lib])

    result = 0
    for book in SCANED_BOOKS:
        result += book.score
    return result

In [8]:
class TimeoutException(Exception):
    pass

def timeout_handler(signum,frame):
    raise TimeoutException

Handler = signal.signal(signal.SIGALRM,timeout_handler)

AttributeError: module 'signal' has no attribute 'SIGALRM'

In [156]:
for file in os.listdir('.'):
    if not file.endswith('.txt'):
	    continue
    LIBRARIES,NUM_OF_BOOKS, NUM_OF_LIBS, MAX_TIME = read_from_file(file)
    signal.alarm(300)
    try:
        start = time.time()
        result = solve(set(),set())
        ti = time.time() - start
        print("Dataset: {}\nThe result achieved: {} in: {}s".format(file,result,round(ti,4)))
    except TimeoutException:
        print("Dataset: {}\nTimeout".format(file))
    signal.alarm(0)
    signal.signal(signal.SIGALRM,Handler)

  2%|▏         | 16/1000 [00:04<05:04,  3.24it/s]


Dataset: a_example.txt
The result achieved: 21 in: 0.0034s
Dataset: c_incunabula.txt
The result achieved: 5689822 in: 116.593s
Dataset: e_so_many_books.txt
The result achieved: 5026488 in: 33.608s
Dataset: d_tough_choices.txt
Timeout
Dataset: b_read_on.txt
The result achieved: 5822900 in: 2.128s
Dataset: f_libraries_of_the_world.txt
The result achieved: 5216624 in: 4.9494s
